In [1]:
import os
print(os.getcwd())

C:\Users\u5819317\Desktop\Python for QuantEco\LifeCycle_0_1


In [3]:
os.chdir('C:\\Users\\u5819317\\Desktop\\Python for QuantEco\\LifeCycle_0_1')
print(os.getcwd())

C:\Users\u5819317\Desktop\Python for QuantEco\LifeCycle_0_1


In [10]:
import pandas as pd
interestTable = pd.read_csv('return_H.csv', usecols=[1,2,3])  ## Read in interest rate
mortalityTable = pd.read_csv('newmort2.csv', usecols=[1,2,3])  ## Read in mortality table

import numpy as np
from datascience import Table
Expectation = Table.from_df(interestTable)  ## Convert readin csv file to 'table'

In [39]:
def func(θ):
    Expect1 = 1 + θ*Expectation.column('Eq') + (1-θ)*Expectation.column('Bond')
    Expect2 = np.power(Expect1, -5) ### How to wrap this into the class?? Now the rho is 5.
    Expect3 = Expectation.column('Eq') - Expectation.column('Bond')
    Expect4 = Expectation.column('Probability') * (Expect2*Expect3)
    return Expect4.sum()

from scipy.optimize import root

In [43]:
class optim:
    
    def __init__(self, ρ = 5, 
                       grids = 50, 
                       BalLow = 20000.0, 
                       BalHigh = 1000000.0):
        self.ρ = ρ
        self.grids = grids
        self.BL = BalLow
        self.BH = BalHigh
    
    
    def Optimal_109(self):
        
        M109 = np.arange(self.BL, self.BH+1, int(self.BH/self.grids))
        
        
        sol = root(func, 0.5)  ## 0.5 is the starting point of θ
        θ109 = sol.x    # sol.fun is the value of the objective equation, very close to zero
                        ## Should add constraint [0,1] on θ in further calculation.
        Expect1 = 1 + θ109 * Expectation.column('Eq') + (1-θ109) * Expectation.column('Bond')
        Expect5 = np.power(Expect1, 1-self.ρ) * Expectation.column('Probability')
        C109 = np.power((1-mortalityTable['X'][108]) * np.power(M109, -self.ρ) * Expect5.sum(), -1/self.ρ) 
        ## change 108 to [age-1] when do the iteration
        return C109, θ109
    

In [44]:
op = optim()
(C109, θ109) = op.Optimal_109()

print('Optimal risky assets allocation rate at 109 is ' + str(θ109))
print('Optimal consumption at 109 is ' + str(C109))

Optimal risky assets allocation rate at 109 is [0.48775782]
Optimal consumption at 109 is [  22808.87801202   45617.75602404   68426.63403605   91235.51204807
  114044.39006009  136853.26807211  159662.14608413  182471.02409615
  205279.90210816  228088.78012018  250897.6581322   273706.53614422
  296515.41415624  319324.29216826  342133.17018027  364942.04819229
  387750.92620431  410559.80421633  433368.68222835  456177.56024037
  478986.43825238  501795.3162644   524604.19427642  547413.07228844
  570221.95030046  593030.82831248  615839.70632449  638648.58433651
  661457.46234853  684266.34036055  707075.21837257  729884.09638459
  752692.9743966   775501.85240862  798310.73042064  821119.60843266
  843928.48644468  866737.3644567   889546.24246871  912355.12048073
  935163.99849275  957972.87650477  980781.75451679 1003590.6325288
 1026399.51054082 1049208.38855284 1072017.26656486 1094826.14457688
 1117635.0225889  1140443.90060091]


In [71]:
from scipy.interpolate import interp1d

class optim:
    
    def __init__(self, ρ = 5, 
                       Z = 0, 
                       grids = 50, 
                       BalLow = 20000.0, 
                       BalHigh = 1000000.0):
        self.ρ = ρ
        self.grids = grids
        self.BL = BalLow
        self.BH = BalHigh
        self.Z = Z
    
    def func(self, θ):
        Expect1 = 1 + θ*Expectation.column('Eq') + (1-θ)*Expectation.column('Bond')
        Expect2 = np.power(Expect1, -5) ### How to wrap this into the class?? Now the rho is 5.
        Expect3 = Expectation.column('Eq') - Expectation.column('Bond')
        Expect4 = Expectation.column('Probability') * (Expect2*Expect3)
        return Expect4.sum()

    def Optimal_109(self):
        
        M109 = np.arange(self.BL, self.BH+1, int(self.BH/self.grids))
        
        sol = root(func, 0.5)  ## 0.5 is the starting point of θ
        θ109 = sol.x    # sol.fun is the value of the objective equation, very close to zero
                        ## Should add constraint [0,1] on θ in further calculation.
        Expect1 = 1 + θ109 * Expectation.column('Eq') + (1-θ109) * Expectation.column('Bond')
        Expect5 = np.power(Expect1, 1-self.ρ) * Expectation.column('Probability')
        C109 = np.power((1-mortalityTable['X'][108]) * np.power(M109, -self.ρ) * Expect5.sum(), -1/self.ρ) 
        ## change 108 to [age-1] when do the iteration
        return C109, θ109
    
    def Optimal_108(self):
        ρ = self.ρ
        
        M109 = np.arange(self.BL, self.BH+1, int(self.BH/self.grids))
        M108 = M109
        A109 = M109 - self.Z + C109
        Optimal_results = Table().with_columns([ ## Collect the results (optimal consumption Cx \
            'M109', M109,                             ## and the corresponding Ax/Mx) into a table
            'C109', C109,
            'A109', A109,
        ])
        
        def theta(x):
            Interpo1 = 1 + (1-x)*Expectation.column('Bond') + x*Expectation.column('Eq')
            Interpo2 = node * Interpo1 ## A109_Interpo given M108 (node)
    
            Interpolation = interp1d(A109, C109, kind='linear', fill_value='extrapolate')
            Interpo3 = Interpolation(Interpo2) / Optimal_results.column(1)[M108==node] 
    
            Interpo4 = np.power(Interpo3, -ρ) * (Expectation.column('Eq') - Expectation.column('Bond'))
            Interpo5 = Expectation.column('Probability') * Interpo4
            return(Interpo5.sum())
        
        θ108 = []
        for node in M108:
            solution = root(theta, 0.5)
            θ = float(solution.x)
            if θ <= 0: ## Constraint
                θ = 0
            elif θ >= 1:
                θ = 1
            θ108.append(θ)
        
        C108 = []
        for index, node in enumerate(M108):
            cal1 = 1 + (1-θ108[index])*Expectation.column('Bond') + θ108[index]*Expectation.column('Eq')
            cal2 = node * cal1    
            Interpolation = interp1d(A109, C109, kind='linear', fill_value='extrapolate')
            cal3 = Interpolation(cal2)
            cal4 = np.power(cal3, -ρ) * cal1 * Expectation.column('Probability')
            cal5 = cal4.sum()
            cal6 = np.power(cal5 * (1-mortalityTable['X'][107]), -1/ρ)
            C108.append(cal6)
        
        return C108, θ108
        
        

In [78]:
op = optim()
a = op.Optimal_108()

In [79]:
print(a[0])

[12109.693227108368, 24219.386454216736, 36329.07968132511, 48438.77290843347, 60548.46613554185, 72658.15936265021, 84767.85258975858, 96877.54581686696, 108987.23904397532, 121096.93227108369, 133206.62549819206, 145316.31872530043, 157426.0119524088, 169535.7051795172, 181645.39840662555, 193755.09163373391, 205864.78486084228, 217974.47808795064, 230084.171315059, 242193.8645421674, 254303.55776927577, 266413.25099638413, 278522.9442234925, 290632.6374506009, 302742.33067770925, 314852.02390481764, 326961.717131926, 339071.4103590344, 351181.1035861427, 363290.7968132511, 375400.4900403595, 387510.18326746783, 399619.8764945762, 411729.56972168456, 423839.26294879295, 435948.95617590134, 448058.6494030097, 460168.3426301181, 472278.03585722647, 484387.72908433486, 496497.4223114432, 508607.1155385516, 520716.80876566, 532826.5019927683, 544936.1952198767, 557045.888446985, 569155.5816740935, 581265.2749012018, 593374.9681283102, 605484.6613554185]


In [80]:
print(a[1])

[0.487757823118668, 0.4877578231186681, 0.4877578231186681, 0.48775782311866805, 0.487757823118668, 0.487757823118668, 0.48775782311866817, 0.4877578231186682, 0.48775782311866817, 0.487757823118668, 0.4877578231186681, 0.487757823118668, 0.487757823118668, 0.48775782311866805, 0.48775782311866817, 0.4877578231186683, 0.4877578231186681, 0.4877578231186681, 0.4877578231186682, 0.4877578231186682, 0.48775782311866805, 0.4877578231186679, 0.48775782311866794, 0.487757823118668, 0.487757823118668, 0.487757823118668, 0.48775782311866817, 0.4877578231186681, 0.48775782311866817, 0.48775782311866817, 0.4877578231186682, 0.4877578231186683, 0.48775782311866817, 0.48775782311866805, 0.4877578231186681, 0.487757823118668, 0.487757823118668, 0.48775782311866794, 0.48775782311866783, 0.487757823118668, 0.48775782311866794, 0.487757823118668, 0.4877578231186679, 0.487757823118668, 0.48775782311866817, 0.487757823118668, 0.4877578231186682, 0.48775782311866817, 0.48775782311866817, 0.48775782311866